In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

# XGBoosting (extreme Gradient Boosting)

## 可看成 Random Forest 強化版

- XGBoost以“正則化提升(regularized boosting)”技術而聞名
- XGBoost可以實現並行處理，相比GBM有了速度的飛躍。
- XGBoost 也支持Hadoop實現。
- XGBoost內置處理缺失值的規則。
####    用戶需要提供一個和其它樣本不同的值，然後把它作為一個參數傳進去，以此來作為缺失值的取值。XGBoost在不同節點遇到缺失值時  #### 采用不同的處理方法，並且會學習未來遇到缺失值時的處理方法。
<br>
- XGBoost會一直分裂到指定的最大深度(max_depth)，然後回過頭來剪枝。如果某個節點之後不再有正值，它會去除這個分裂。
- XGBoost允許在每一輪boosting叠代中使用交叉驗證。因此，可以方便地獲得最優boosting叠代次數。
- XGBoost可以在上一輪的結果上繼續訓練。這個特性在某些特定的應用上是一個巨大的優勢。



# Parameter

## 1. 通用參數

- booster: 選擇每次叠代的模型，有兩種選擇：gbtree：基於樹的模型 , gbliner：線性模型。 gbtree表現遠勝過Linear
- silent: 當這個參數值為1時，靜默模式開啟，不會輸出任何信息。  default = 0
- nthread:如果你希望使用CPU全部的核，那就不要輸入這個參數

## 2. Booster 參數 (for gbtree)

- eta: 和GBM中的 learning rate 參數類似。 典型值為0.01-0.2。
- min_child_weight: 決定最小葉子節點樣本權重和。 use to avoid overfitting，數值過大 >> Underfitting, 過小: overfit。 
#### 需靠 CV 調整
<br>
- max_depth: 樹的最大深度，big >> overfit, small >> underfit。 典型值：3-10
- max_leaf_nodes: 樹上最大的節點或葉子的數量，可以替代max_depth的作用。因為如果生成的是二叉樹，一個深度為n的樹最多生成n2個葉子。
- gamma: 指定了節點分裂所需的最小損失函數下降值。參數的值越大，算法越保守。 default = 0, 需要調整
- max_delta_step: 限制每棵樹權重改變的最大步長，通常，這個參數不需要設置。但是當各類別的樣本十分不平衡時，它對邏輯回歸是很有幫助的。
- subsample: 控制對於每棵樹隨機采樣的"比例"，big >>　overfit, small >> underfit。 典型值：0.5-1, default = 1
- lambda: L2 norm. default = 1, 少用到這個參數
- alpha: L1 norm. default = 1. 可以應用在很高維度的情況下，使得算法的速度更快
- scale_pos_weight: 在各類別樣本十分不平衡時，把這個參數設定為一個正值，可以使算法更快收斂。

## 3. 學習目標參數: 控制理想的優化目標和每一步結果的度量方法

- objective: 這個參數定義需要被最小化的損失函數。最常用的值有
1. binary:logistic  二分類的邏輯回歸，返回預測的概率(不是類別)。
2. multi:softmax   使用softmax的多分類器，返回預測的類別(不是概率)。在這種情況下，你還需要多設一個參數：num_class(類別數目)。
3. multi:softprob   和multi:softmax參數一樣，但是返回的是每個數據屬於各個類別的概率。
<br>
- eval_metric:[默認值取決於objective參數的取值], 對於有效數據的度量方法。 對於回歸問題，默認值是rmse，對於分類問題，默認值是error。
1. 選擇: rmse, mae, logloss, error(二分類錯誤率(閾值為0.5)),merror (多分類錯誤率), mlogloss (多分類logloss損失函數), auc
- seed:  default = 0

# Sklearn 中有XGBClassifier 

### from xgboost.sklearn import XGBClassifier

- eta --> learning_rate
- lambda --> reg_lambda
- alpha --> reg_alpha

In [4]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional     scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.model_selection import cross_validate

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

#train = pd.read_csv('train_modified.csv',encoding = 'ISO-8859-1')
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')
target = 'Disbursed'
IDcol = 'ID'

## XGBoost

詳細說明: 
         <p>(ENG) https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/</p>
         <p>(CHT) http://www.itread01.com/articles/1476146171.html</p>

- xgb - 直接引用xgboost。接下來會用到其中的“cv”函數。
- XGBClassifier - 是xgboost的sklearn包。這個包允許我們像GBM一樣使用Grid Search 和並行處理


## 定義一個函數，可以建立XGBoost models 並進行交叉驗證。以後再自己的models中也可以使用

In [14]:
result

NameError: name 'result' is not defined

In [1]:
def modelfit(alg, dtrain,dtest,predictors,useTrainCV = True,cv_folds = 5, early_stopping_rounds = 50):
     
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label = dtrain[target].values)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = alg.get_params()['n_estimators'],nfold = cv_folds,
                         metrics = 'auc', early_stopping_rounds = early_stopping_rounds, show_stdv=False )
        ### assign best n_estimator to xgboost
        alg.set_params(n_estimators = cvresult.shape[0])
        
        
    ## Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric = 'auc')
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Print ,model report:
    print("\nModel Report")
    print("Accuracy: %4g" % metrics.accuracy_score(dtrain['Disbursed'].values,dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'].values, dtrain_predprob))
#     Predict on testing data:
    dtest['predprob'] = alg.predict_proba(dtest[predictors])[:,1]
    results = test_results.merge(dtest[['ID','predprob']], on='ID')
    print ('AUC Score (Test): %f' % metrics.roc_auc_score(results['Disbursed'], results['predprob']))    
    
    print("best n_estimators = ", cvresult.shape[0])
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending = False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel("Feature Importance Score")

# 實作指南

#### 使用和GBM中相似的方法。需要進行如下步驟： 
-  選擇較高的學習速率(learning rate)。一般情況下，學習速率的值為0.1。但是，對於不同的問題，理想的學習速率有時候會在0.05到0.3之間波動。選擇對應於此學習速率的理想決策樹數量。XGBoost有一個很有用的函數“cv”，這個函數可以在每一次叠代中使用交叉驗證，並返回理想的決策樹數量。 
-  對於給定的學習速率和決策樹數量，進行決策樹特定參數調優(max_depth, min_child_weight, gamma, subsample, colsample_bytree)。在確定一棵樹的過程中，我們可以選擇不同的參數。
-  xgboost的正則化參數的調優。(lambda, alpha)。這些參數可以降低模型的復雜度，從而提高模型的表現。 
-  降低學習速率，確定理想參數。




## 1. 確定learning rate和tree_based 參數調優的estimator數目。

### 為了確定boosting 參數，要先給其它參數一個初始值。先按如下方法取值： 
- 1、max_depth = 5 :這個參數的取值最好在3-10之間。我選的起始值為5，但是你也可以選擇其它的值。起始值在4-6之間都是不錯的選擇。 
- 2、min_child_weight = 1:在這裏選了一個比較小的值，因為這是一個"極不平衡的分類問題"。因此，某些葉子節點下的值會比較小。 
- 3、gamma = 0: 起始值也可以選其它比較小的值，在0.1到0.2之間就可以。這個參數後繼也是要調整的。 
- 4、subsample,colsample_bytree = 0.8: 這個是最常見的初始值了。典型值的範圍在0.5-0.9之間。 
- 5、scale_pos_weight = 1: 這個值是因為類別十分不平衡。 

#### 註意哦，上面這些參數的值只是一個初始的估計值，後繼需要調優。這裏把學習速率就設成默認的0.1。然後用xgboost中的cv函數來確定最佳的決策樹數量。前文中的函數可以完成這個工作。

In [2]:
predictors = [x for x in train.columns if x not in [target,IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators = 1000,
 max_depth = 5,
 min_child_weight= 1,
 gamma = 0,
 subsample = 0.8,
 colsample_bytree = 0.8,
 objective = 'binary:logistic',
 nthread = 4,
 scale_pos_weight = 1,
 seed = 27)

modelfit(xgb1,train,test,predictors)

NameError: name 'train' is not defined

#### 在learning_rate = 0.1 時, best n_estimator = 112   >> 使用cv函數可以看到在指定learning_rate下的最佳n_estimators 

# 第二步： max_depth 和 min_weight 參數調優

- 兩個參數最終結果有很大的影響。首先，我們先大範圍地粗調參數，然後再小範圍地微調。 
註意：在這一節我會進行高負荷的柵格搜索(grid search)，這個過程大約需要15-30分鐘甚至更久，具體取決於你系統的性能。你也可以根據自己系統的性能選擇不同的值。

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':list(range(3,10,2)),
    'min_child_weight':list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=112, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_,     gsearch1.best_score_

#### 至此，我們對於數值進行了較大跨度的12中不同的排列組合，可以看出理想的max_depth值為5，理想的min_child_weight值為3 (和說明文件有落差)。在這

#### 個值附近我們可以再進一步調整，來找出理想值。我們把上下範圍各拓展1，因為之前我們進行組合的時候，參數調整的步長是2。

In [3]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test2 = {
    'max_depth':[4,5,6],
    'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
                                        min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])

gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

NameError: name 'GridSearchCV' is not defined

# 第三步：gamma參數調優

- 在已經調整好其它參數的基礎上，我們可以進行gamma參數的調優了。Gamma參數取值範圍可以很大，我這裏把取值範圍設置為5了。你其實也可以取更精確的 gamma值。



In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

####  從這裏可以看出來，我們在第一步調參時設置的初始gamma值就是比較合適的。也就是說，理想的gamma值為0。在這個過程開始之前，最好重新調整boosting回合，因為參數都有變化。


# 使用得到的參數重跑

In [ ]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
scale_pos_weight=1,
seed=27)
modelfit(xgb2, train,test, predictors)

# 第四步：調整subsample 和 colsample_bytree 參數

- 下一步是嘗試不同的subsample 和 colsample_bytree 參數。我們分兩個階段來進行這個步驟。這兩個步驟都取0.6,0.7,0.8,0.9作為起始值。

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=3,
 min_child_weight=4, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

#### 現在，colsample_bytree = 0.8, subsample = 0.9，我們以0.05為步長，在這個值附近嘗試取值。



In [ ]:
param_test5 = {
 'subsample':[i/100.0 for i in range(80,100,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

#### 我們得到的理想取值還是原來的值。因此，最終的理想取值是:

- subsample: 0.8
- colsample_bytree: 0.9

# 第五步：正則化參數調優。

#### 下一步是應用正則化來降低過擬合。由於gamma函數提供了一種更加有效地降低過擬合的方法，大部分人很少會用到這個參數。但是我們在這裏也可以嘗試用一下這個參數。我會在這裏調整’reg_alpha’參數，然後’reg_lambda’參數留給你來完成。

In [ ]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors],train[target])
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

#### 我們可以看到，相比之前的結果，CV的得分甚至還降低了。但是我們之前使用的取值是十分粗糙的，我們在這裏選取一個比較靠近理想值(0.01)的取值，來看看是否有更好的表現。

In [ ]:
param_test7 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.9,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(train[predictors],train[target])
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

#### CV的得分沒提高 >> 不使用 rel_alpha

# 第6步：降低學習速率

- 最後，我們使用較低的學習速率，以及使用更多的決策樹。我們可以用XGBoost中的CV函數來進行這一步工作。

In [ ]:
xgb4 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.9,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb4, train,test, predictors)

### 至此，你可以看到模型的表現有了大幅提升，調整每個參數帶來的影響也更加清楚了。 在文章的末尾，我想分享兩個重要的思想： 
- 1、僅僅靠參數的調整和模型的小幅優化，想要讓模型的表現有個大幅度提升是不可能的。GBM的最高得分是0.8487，XGBoost的最高得分是0.8494。確實是 有一定的提升，但是沒有達到質的飛躍。 
<br>

- 2、要想讓模型的表現有一個質的飛躍，需要依靠其他的手段，諸如，特征工程(feature egineering) ，模型組合(ensemble of model),以及堆疊
  (stacking)等。